In [ ]:
'sk-proj-G2Z8UDRc4T4hx7LVmjsAT3BlbkFJSKSOS42WX8vWXqow0jGn'



In [12]:
pip install faiss-cpu
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [2]:
# from lang_funcs import *
from langchain.llms import Ollama
from langchain import PromptTemplate
# Importing the necessary packages
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap

In [3]:
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )


# Function for creating embeddings using FAISS
def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    
    # Saving the model in current directory
    vectorstore.save_local(storing_path)
    
    # returning the vectorstore
    return vectorstore
# This will load the PDF file
def load_pdf_data(file_path):
    # Creating a PyMuPDFLoader object with file_path
    loader = PyMuPDFLoader(file_path=file_path)
    
    # loading the PDF file
    docs = loader.load()
    
    # returning the loaded document
    return docs
# Responsible for splitting the documents into several chunks
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    
    # Initializing the RecursiveCharacterTextSplitter with
    # chunk_size and chunk_overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents=documents)
    
    # returning the document chunks
    return chunks
# Creating the chain for Question Answering
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
        chain_type_kwargs={'prompt': prompt} # customizing the prompt
    )
# Prettifying the response
def get_response(query, chain):
    # Getting response from chain
    response = chain({'query': query})
    
    # Wrapping the text for better output in Jupyter Notebook
    wrapped_text = textwrap.fill(response['result'], width=100)
    print(wrapped_text)

In [4]:
# Loading orca-mini from Ollama
llm = Ollama(model="mistral", temperature=0)

# Loading the Embedding Model
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

D:\python_enviroments\gen_env_3.11.5\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\python_enviroments\gen_env_3.11.5\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
# loading and splitting the documents
docs = load_pdf_data(file_path="clawset_doc.pdf")
documents = split_docs(documents=docs)

# creating vectorstore
vectorstore = create_embeddings(documents, embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

In [18]:
template = """
### System:
You are an respectful and honest assistant. You have to answer the user's \
questions using only the context provided to you. If you don't know the answer, \
just say you don't know. Don't try to make up an answer.

### Context:
{context}

### User:
{question}

### Response:
"""


# template = """
# ### System:
# You are an respectful and honest assistant. You have to answer the questions related to the clawset website only. If you don't know the answer, \
# just say you don't know. Don't try to make up an answer.

# ### Context:
# {context}

# ### User:
# {question}

# ### Response:
# """

prompt = """
### System:
You are an AI Assistant that follows instructions extreamly well. \
Help as much as you can.

### User:
{prompt}

### Response:

"""

In [19]:
# Creating the prompt from the template which we created before
prompt = PromptTemplate.from_template(template)

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [20]:
get_response("give me home page link of clawset website", chain)

 The homepage link of the Clawset website is http://clawset.co/.


In [21]:
get_response("give me new products name of clawset website and give me link of page too", chain)

 The latest additions to the Clawset product lineup include "Pet Collars – Pantone Colors", "Collar
set with leash, Pantone", "Leather Collar – Pastel", and "Training Leash – Marshmallow". You can
find these new products on the following page: http://clawset.co/new-product/


In [22]:
get_response("who is elon musk?", chain)

 Elon Musk is a South African-born American entrepreneur and business magnate. He is the CEO of
SpaceX, CEO of Neuralink, and the founder of The Boring Company and Tesla Inc. He is known for his
innovative ideas in renewable energy, electric vehicles, and space travel. However, he is not
related to the Clawset website or its products.


# Type: 2

In [1]:


import fitz  # PyMuPDF
from textwrap import dedent
from typing import List
from fastembed import TextEmbedding
from groq import Groq
from groq.types.chat import ChatCompletionMessage
import pandas as pd
import numpy as np

# Set up the Groq API key and client
import os
os.environ["GROQ_API_KEY"]="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"
client = Groq()

MODEL = "llama-3.1-70b-versatile"
TEMPERATURE = 0
MAX_TOKENS = 4096

# Read text from PDF
def read_pdf(file_path: str) -> str:
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load the PDF
pdf_text = read_pdf("clawset_doc.pdf")

# Prepare the knowledge base from PDF text
KNOWLEDGE_BASE = [
    {"question": "Extracted from PDF", "answer": pdf_text}
]

# # Initialize the embedding model
# embedding_model = TextEmbedding()

# # Create embeddings for the text from the PDF
# document_embeddings = list(embedding_model.embed(KNOWLEDGE_BASE[0]["question"]))
# print(f"Embedding shape: {document_embeddings[0].shape}")


# # def retrieve_context(query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model) -> str:
# #     query_embedding = list(embedding_model.embed([query]))[0]
# #     # Assuming you have a DataFrame called 'data' with the relevant columns
# #     results_df = data.merge(
# #         document_embeddings,
# #         left_on='id',
# #         right_on='id',
# #         how='left'
# #     ).query(f"embedding MATCH {serialize_float32(query_embedding)}")
    
# #     results_df.sort_values(by='distance', inplace=True)
# #     return "\n---\n".join([f"{item['question']}\n{item['answer']}" for _, item in results_df.iterrows()])



# def retrieve_context(query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model) -> str:
#     query_embedding = list(embedding_model.embed([query]))[0]
#     results = db.execute(
#         """
#         SELECT
#             document_embeddings.id,
#             distance,
#             question,
#             answer
#         FROM document_embeddings
#         LEFT JOIN documents ON documents.id = document_embeddings.id
#         WHERE embedding MATCH ?
#             AND k = ?
#         ORDER BY distance
#         """,
#         [serialize_float32(query_embedding), k],
#     ).fetchall()
#     return "\n---\n".join([f"{item[2]}\n{item[3]}" for item in results])


# Initialize embedding model
embedding_model = TextEmbedding()

# Convert KNOWLEDGE_BASE to a DataFrame
documents_df = pd.DataFrame(KNOWLEDGE_BASE)
documents_df['id'] = documents_df.index

# Create a text column combining question and answer
documents_df['text'] = documents_df['question'] + '\n' + documents_df['answer']

# Embed documents and convert embeddings to lists
document_embeddings = list(embedding_model.embed(documents_df['text'].tolist()))
embedding_dim = document_embeddings[0].shape[0]  # Assuming all embeddings have the same dimension
documents_df['embedding'] = [embedding.tolist() for embedding in document_embeddings]

# Function to retrieve context based on query
def retrieve_context(query: str, k: int = 3) -> str:
    # Embed the query and convert to list
    query_embedding = list(embedding_model.embed([query]))[0].tolist()
    
    # Compute distances between the query embedding and document embeddings
    distances = np.linalg.norm(
        np.array(documents_df['embedding'].tolist()) - np.array(query_embedding),
        axis=1
    )
    
    # Add distances to the DataFrame
    documents_df['distance'] = distances
    
    # Sort DataFrame by distance and get top-k results
    top_k_df = documents_df.nsmallest(k, 'distance')
    
    # Prepare the result
    result = "\n---\n".join([f"{row['question']}\n{row['answer']}" for _, row in top_k_df.iterrows()])
    return result

query = "What is the warranty for the parts?"
print(retrieve_context(query))

SYSTEM_PROMPT = """
You're a senior customer support agent for an online autoparts store.
You're always helpful and answer customer questions only based on the provided
information. If you don't know the answer - just reply with an excuse that you
don't know. Keep your answers brief and to the point. Be kind and respectful.

Use the provided context for your answers. The most relevant information is
at the top. Each piece of information is separated by ---.
"""
def create_user_prompt(query: str) -> str:
    return dedent(
        f"""
Use the following information:

```
{retrieve_context(query)}
```

to answer the question:
{query}
    """
    )
print(create_user_prompt(query))

user_prompt = create_user_prompt(query)

messages = [    {        "role": "system",        "content": SYSTEM_PROMPT,    },]

def call_model(query: str, messages: List) -> ChatCompletionMessage:
    messages.append(
        {
            "role": "user",
            "content": create_user_prompt(query),
        },
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
    )
    message = response.choices[0].message
    messages.append(message)
    return message

query = "What is the warranty for the parts?"
print('question:', query)

message = call_model(query, messages)
print(message.content)

query = "Can I return a part that I incorrectly ordered?"
print('question:', query)
message = call_model(query, messages)
print(message.content)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracted from PDF
Main Pages 
1. Homepage 
o URL: http://clawset.co/ 
o Description: The main landing page featuring highlights of their product 
offerings and special collections. 
Because you can choose the best and most suitable for both yourself and your 
pet, CLAWSET BKK has a slogan of ' My Pet My Style' 
to allow all dog and cat parents and owners to choose the right style of pet 
clothing in the best quality with us. And with this slogan, CLAWSET BKK 
has designed a collection of pet clothing,  dog clothing , cat clothing , and pet 
matching shirts for everyone to be able to own dog and cat clothing that is 
designed to stand out, modern, beautiful, and stylish every day. 
Collection of pet sets, pet couple shirts, exclusive copyright at CLAWSET 
BKK 
For those who are looking for pet clothes, pet shirts that match their owners, 
and pet fashion that has a unique and distinctive style, we would like to 
introduce everyone to the CLAWSET BKK pet clothing collection that has a 


In [7]:
query = "what are new products?"
print('question:', query)
message = call_model(query, messages)
print(message.content)

question: what are new products?
Our new products include:

1. Pet Collars – Pantone Colors
2. Collar set with leash, Pantone
3. Leather Collar – Pastel
4. Training Leash – Marshmallow

You can find more information about our new products on our New Products page: http://clawset.co/new-product/.


# using text

In [29]:


import fitz  # PyMuPDF
from textwrap import dedent
from typing import List
from fastembed import TextEmbedding
from groq import Groq
from groq.types.chat import ChatCompletionMessage
import pandas as pd
import numpy as np
import re
# Set up the Groq API key and client
import os
os.environ["GROQ_API_KEY"]="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"
client = Groq()

MODEL = "llama-3.1-70b-versatile"
TEMPERATURE = 0
MAX_TOKENS = 4096

# Read text from PDF
def read_pdf(file_path: str) -> str:
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load the PDF
pdf_text = read_pdf("user_data.pdf")
# f = open("user_data.txt", "r", encoding="utf8")
# data = f.read()
# data = re.sub('Human 2.*?\n', '', data)
# data = data.replace("\n", ". ")
# pdf_text = data.replace("Human 1: ", "")

# Prepare the knowledge base from PDF text
KNOWLEDGE_BASE = [
    {"question": "Extracted from PDF", "answer": pdf_text}
]

# # Initialize the embedding model
# embedding_model = TextEmbedding()

# # Create embeddings for the text from the PDF
# document_embeddings = list(embedding_model.embed(KNOWLEDGE_BASE[0]["question"]))
# print(f"Embedding shape: {document_embeddings[0].shape}")


# # def retrieve_context(query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model) -> str:
# #     query_embedding = list(embedding_model.embed([query]))[0]
# #     # Assuming you have a DataFrame called 'data' with the relevant columns
# #     results_df = data.merge(
# #         document_embeddings,
# #         left_on='id',
# #         right_on='id',
# #         how='left'
# #     ).query(f"embedding MATCH {serialize_float32(query_embedding)}")
    
# #     results_df.sort_values(by='distance', inplace=True)
# #     return "\n---\n".join([f"{item['question']}\n{item['answer']}" for _, item in results_df.iterrows()])



# def retrieve_context(query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model) -> str:
#     query_embedding = list(embedding_model.embed([query]))[0]
#     results = db.execute(
#         """
#         SELECT
#             document_embeddings.id,
#             distance,
#             question,
#             answer
#         FROM document_embeddings
#         LEFT JOIN documents ON documents.id = document_embeddings.id
#         WHERE embedding MATCH ?
#             AND k = ?
#         ORDER BY distance
#         """,
#         [serialize_float32(query_embedding), k],
#     ).fetchall()
#     return "\n---\n".join([f"{item[2]}\n{item[3]}" for item in results])


# Initialize embedding model
embedding_model = TextEmbedding()

# Convert KNOWLEDGE_BASE to a DataFrame
documents_df = pd.DataFrame(KNOWLEDGE_BASE)
documents_df['id'] = documents_df.index

# Create a text column combining question and answer
documents_df['text'] = documents_df['question'] + '\n' + documents_df['answer']

# Embed documents and convert embeddings to lists
document_embeddings = list(embedding_model.embed(documents_df['text'].tolist()))
embedding_dim = document_embeddings[0].shape[0]  # Assuming all embeddings have the same dimension
documents_df['embedding'] = [embedding.tolist() for embedding in document_embeddings]

# Function to retrieve context based on query
def retrieve_context(query: str, k: int = 3) -> str:
    # Embed the query and convert to list
    query_embedding = list(embedding_model.embed([query]))[0].tolist()
    
    # Compute distances between the query embedding and document embeddings
    distances = np.linalg.norm(
        np.array(documents_df['embedding'].tolist()) - np.array(query_embedding),
        axis=1
    )
    
    # Add distances to the DataFrame
    documents_df['distance'] = distances
    
    # Sort DataFrame by distance and get top-k results
    top_k_df = documents_df.nsmallest(k, 'distance')
    
    # Prepare the result
    result = "\n---\n".join([f"{row['question']}\n{row['answer']}" for _, row in top_k_df.iterrows()])
    return result

query = "What is the warranty for the parts?"
print(retrieve_context(query))

# SYSTEM_PROMPT = """
# You're a senior customer support agent for an online autoparts store.
# You're always helpful and answer customer questions only based on the provided
# information. If you don't know the answer - just reply with an excuse that you
# don't know. Keep your answers brief and to the point. Be kind and respectful.

# Use the provided context for your answers. The most relevant information is
# at the top. Each piece of information is separated by ---.
# """


SYSTEM_PROMPT = """
You're a senior customer support agent for each user's data.
You're always helpful and answer customer questions only based on the provided
information. If user ask something about his future, give the user some suggestions based on its previous data. If you don't know the answer - just reply with an excuse that you
don't know. Keep your answers brief and to the point. Be kind and respectful.

Use the provided context for your answers. The most relevant information is
at the top. Each piece of information is separated by ---.
"""

def create_user_prompt(query: str) -> str:
    return dedent(
        f"""
Use the following information:

```
{retrieve_context(query)}
```

to answer the question:
{query}
    """
    )
print(create_user_prompt(query))

user_prompt = create_user_prompt(query)

messages = [    {        "role": "system",        "content": SYSTEM_PROMPT,    },]

def call_model(query: str, messages: List) -> ChatCompletionMessage:
    messages.append(
        {
            "role": "user",
            "content": create_user_prompt(query),
        },
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
    )
    message = response.choices[0].message
    messages.append(message)
    return message

# query = "What is the warranty for the parts?"
# print('question:', query)

# message = call_model(query, messages)
# print(message.content)



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracted from PDF
03/02/2024 
Hi! 
04/02/2024 
one where I get to meet lots of different people. 
05/02/2024 
Hard to keep a count. Maybe 25. 
06/02/2024 
I think it was Australia 
07/02/2024 
Not really. The interactions are usually short-lived bu
t it's fascinating to learn where people are coming fro
m and what matters to them 
08/02/2024 
what do you mean? 
09/02/2024 
Wow! That's interesting, borderline spooky 
10/02/2024 
So, do you do treasure hunting often? 
11/02/2024 
Hi 
12/02/2024 
my friends are gonna visit me this weekend. we migh
t go hiking! 
13/02/2024 
Should be very sunny! you? 
14/02/2024 
😐 hope you get your work done very soon! a bug fre
e weekend! 
15/02/2024 
I am going to Diablo! 
16/02/2024 
hours drive from here. still in bay area 
17/02/2024 
 Actually no idea, but it will take the entire day for th
at. 
18/02/2024 
Hi! 
19/02/2024 
Nothing much, how you doin? 
20/02/2024 
Oh wow! Sounds like fun! When was the last time yo
u had seen this friend? 
21/02/202

In [32]:
query = "why are you using user's data?"
print('question:', query)
message = call_model(query, messages)
print(message.content)

question: why are you using user's data?
I'm using your data to better understand you and provide more personalized support. As a senior customer support agent, my goal is to help answer your questions and provide information that's relevant to your interests and needs. By analyzing your data, I can gain insights into your preferences, habits, and goals, which enables me to offer more effective and tailored support.
